In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Model, T5EncoderModel
import pandas as pd
import torch
from torch import nn
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset

In [2]:
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [3]:
# import sentencepiece

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

In [6]:
model = T5EncoderModel.from_pretrained('google/flan-t5-small').to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [7]:
data = np.load('/kaggle/input/nlpproject/SP-train.npy', allow_pickle = True)

In [8]:
data[2]

{'id': 'SP-0_CR',
 'question': 'A chess team has five players, and each player has one coach. But there are only six participants in the team. How is that possible?',
 'answer': 'Each player shares the same coach.',
 'distractor1': 'Some coaches get a raise.',
 'distractor2': 'Some players are backups and not allowed to play.',
 'distractor(unsure)': 'None of above.',
 'label': 0,
 'choice_list': ['Each player shares the same coach.',
  'Some players are backups and not allowed to play.',
  'Some coaches get a raise.',
  'None of above.'],
 'choice_order': [0, 2, 1, 3]}

In [9]:
data = pd.DataFrame(data)

In [10]:
df_train = data.iloc[:405,:]
df_dev = data.iloc[405:456,:]
df_test = data.iloc[456:507,:]

In [11]:
df_dev.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [12]:
print(len(data)) # 405 51 51

507


In [13]:
len(df_train)

405

In [14]:
df_train

,0
0,"{'id': 'SP-0', 'question': 'Mr. and Mrs. Musta..."
1,"{'id': 'SP-0_SR', 'question': 'The six daughte..."
2,"{'id': 'SP-0_CR', 'question': 'A chess team ha..."
3,"{'id': 'SP-1', 'question': 'A woman shoots her..."
4,"{'id': 'SP-1_SR', 'question': 'An individual s..."
...,...
400,"{'id': 'SP-162_SR', 'question': 'What is the m..."
401,"{'id': 'SP-162_CR', 'question': 'There were 10..."
402,"{'id': 'SP-163', 'question': 'Thomas was caugh..."
403,"{'id': 'SP-163_SR', 'question': 'Despite being..."


In [15]:
for i in df_train.columns:
    print(i)

0


In [16]:
df_train.describe()

,0
count,405
unique,405
top,"{'id': 'SP-163_CR', 'question': 'There was a g..."
freq,1


In [17]:
# correct = []
# list1 = []
# list2 = []
# list3 = []

# for i in data:
#     corr_index = i['label']
#     count = 0
    
#     for j in range(len(i['choice_list'])):
#         if j == corr_index:
#             correct.append(i['question']+" "+i['choice_list'][corr_index])
#         else:
#             if count == 0:
#                 list1.append(i['question']+" "+i['choice_list'][j])
#                 count+=1
#             elif count == 1:
#                 list2.append(i['question']+" "+i['choice_list'][j])
#                 count+=1
#             elif count == 2:
#                 list3.append(i['question']+" "+i['choice_list'][j])
#                 count+=1
        
    

In [18]:
# correct_embeddings = tokenizer(correct, max_length = 128, padding=True, truncation=True, return_tensors = 'pt')
# list1_embeddings = tokenizer(list1, max_length = 128, padding=True, truncation=True, return_tensors = 'pt')
# list2_embeddings = tokenizer(list2, max_length = 128, padding=True, truncation=True, return_tensors = 'pt')
# list3_embeddings = tokenizer(list3, max_length = 128, padding=True, truncation=True, return_tensors = 'pt')

In [19]:
# len(correct_embeddings[0]), len(list1_embeddings[0]), len(list2_embeddings[0]), len(list3_embeddings[0])

In [20]:
class TaskDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[0][idx]
        question = entry['question']
        choices = entry['choice_list']
        label = entry['label']
        choice_order = entry['choice_order']
        correct_index = choice_order.index(label)

        tokenized_choices = [self.tokenizer.encode_plus(  question + "[SEP]" + choice, add_special_tokens = True, max_length = 128, truncation=True, padding='max_length', return_tensors = 'pt') for choice in choices]

        input_ids = torch.cat([choice['input_ids'].unsqueeze(0) for choice in tokenized_choices],dim=0)
        attention_mask = torch.cat([choice['attention_mask'].unsqueeze(0) for choice in tokenized_choices],dim=0)
        labels = torch.zeros(len(choices), dtype=torch.long)
        labels[label] = 1

        input_ids_m = []
        attention_mask_m = []
        labels_m = []

        for i in choice_order:
            input_ids_m.append(input_ids[i])
            attention_mask_m.append(attention_mask[i])
            labels_m.append(labels[i])
            

        return input_ids_m, attention_mask_m, torch.tensor(labels_m)
        
        
        

In [21]:
class TaskModel(nn.Module):
    def __init__(self, model):
        super(TaskModel, self).__init__()
        self.t5 = model
        # self.classifier = nn.Linear(self.t5.config.d_model, 1)
        self.classifier = nn.Sequential(
            nn.Linear(self.t5.config.d_model, 768), 
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(768, 1)
        )
        

    def forward(self, input_ids, attention_mask):
        output = self.t5(input_ids = input_ids, attention_mask = attention_mask, return_dict = True)
        last_hidden_states = output.last_hidden_state[:, 0, :]
        logits = self.classifier(last_hidden_states)
        return logits
        

In [22]:
train_dataset = TaskDataset(df_train, tokenizer)
val_dataset = TaskDataset(df_dev, tokenizer)
test_dataset = TaskDataset(df_test, tokenizer)

In [23]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = False)

In [24]:
train_dataset[3]

([tensor([[   71,  2335,  4279,     7,   160,  2553,     5,    37,    29,   255,
            4532,   376, 22409,    21,   147,   305,   676,     5,  4213,     6,
             255,  5168,     7,   376,     5,   299,   305,   676,   865,     6,
              79,   321,   281,    91,    11,   777,     3,     9,  1627,  2634,
             544,     5,   571,    54,    48,    36,    58,  6306,   134,  8569,
             908,   634,  2335,  2347,     3,     9,   126,  2397,     5,     1,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,    

In [25]:
Modelarch = TaskModel(model)

In [26]:
# from torch.utils.data import DataLoader, random_split

# train_size = int(0.8 * len(traindataset))  
# validation_size = len(traindataset) - train_size  

# train_dataset, validation_dataset = random_split(traindataset, [train_size, validation_size])

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False)


In [27]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [28]:
# logits.squeeze(), labels

In [ ]:
# Training loop
# model.train()
Modelarch.to(device)
for epoch in range(10):  # Number of epochs
    total_loss = 0
    Modelarch.train()
    
    for input_ids, attention_mask, labels in train_loader:
        optimizer.zero_grad()
        logits = []
        for i,j in zip(input_ids, attention_mask):
            #print(i.squeeze(0).shape,j.squeeze(0).shape)
            logits.append(Modelarch(i.squeeze(0).to(device),j.squeeze(0).to(device)))
        logits = torch.stack(logits).squeeze()
        labels = labels.float().squeeze(0).to(device)
        # print(logits)
        # print(labels)
        # logits = Modelarch(input_ids, attention_mask)
        loss = criterion(logits, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    loss = total_loss/len(train_loader)
    print(f'Epoch {epoch+1}, Loss: {loss}')
    
    Modelarch.eval()
    all_predictions = []
    all_labels = []
    total_loss = 0
    
    for input_ids, attention_mask, labels in validation_loader:
            optimizer.zero_grad()
            logits = []
            for i,j in zip(input_ids, attention_mask):
                #print(i.squeeze(0).shape,j.squeeze(0).shape)
                logits.append(Modelarch(i.squeeze(0).to(device),j.squeeze(0).to(device)))
            logits = torch.stack(logits).squeeze()
            labels = labels.float().squeeze(0).to(device)
            # print(logits)
            probabilities = F.softmax(logits, dim=0)
            # Get the predicted class index
            predictions = torch.argmax(probabilities, dim=0)
            predictions = F.one_hot(predictions, num_classes=4) 
            # print(predictions)
            # print(labels)
            # logits = Modelarch(input_ids, attention_mask)
            loss = criterion(logits, labels.float())
            total_loss += loss.item()
            all_predictions.extend(predictions.tolist())
            all_labels.extend(labels.squeeze(0).tolist())

    # Compute accuracy and F1 score
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='macro')
    val_loss = total_loss / len(validation_loader)
    
    print(f"Accuracy: {accuracy}, F1 Score: {f1}, Loss : {val_loss}")

Epoch 1, Loss: 0.6389157368813032
Accuracy: 0.7549019607843137, F1 Score: 0.673202614379085, Loss : 0.6034411121817196
Epoch 2, Loss: 0.5953371986930753
Accuracy: 0.7745098039215687, F1 Score: 0.6993464052287581, Loss : 0.5844644495085174
Epoch 3, Loss: 0.5826148908815266
Accuracy: 0.7156862745098039, F1 Score: 0.6209150326797386, Loss : 0.5779303733040305
Epoch 4, Loss: 0.576270506411423
Accuracy: 0.7843137254901961, F1 Score: 0.7124183006535947, Loss : 0.5731305386505875
Epoch 5, Loss: 0.5619251700831048
Accuracy: 0.8235294117647058, F1 Score: 0.7647058823529411, Loss : 0.5374209810705746


In [ ]:
# # Training loop
# # model.train()
# for epoch in range(5):  # Number of epochs
#     total_loss = 0
#     Modelarch.train()
    
#     for input_ids, attention_mask, labels in train_loader:
#         optimizer.zero_grad()
#         logits = []
#         for i,j in zip(input_ids, attention_mask):
#             #print(i.squeeze(1).shape,j.squeeze(1).shape)
#             logits.append(Modelarch(i.squeeze(1),j.squeeze(1)))
#         #print(logits)
#         logits = torch.stack(logits, dim=2).squeeze(1)
#         labels = labels.float().squeeze(0)
#         #print(logits.shape)
#         #print(labels.shape)
        
#         # logits = Modelarch(input_ids, attention_mask)
#         loss = criterion(logits, labels.float())
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
        
#     loss = total_loss/len(train_loader)
#     print(f'Epoch {epoch+1}, Loss: {loss}')
    
#     Modelarch.eval()
#     all_predictions = []
#     all_labels = []
#     total_loss = 0
    
#     for input_ids, attention_mask, labels in validation_loader:
#             optimizer.zero_grad()
#             logits = []
#             for i,j in zip(input_ids, attention_mask):
#                 # print(i)
#                 # print(i.squeeze(1).shape , j.squeeze(1).shape)
#                 logits.append(Modelarch(i.squeeze(1),j.squeeze(1)))
#             logits = torch.stack(logits, dim=2).squeeze(1)
#             labels = labels.float().squeeze(0)
#             # print(logits)
#             probabilities = F.softmax(logits, dim=0)
#             # Get the predicted class index
#             predictions = torch.argmax(probabilities, dim=1)
#             predictions = F.one_hot(predictions, num_classes=4) 
#             # print(predictions)
#             # print(labels)
#             # logits = Modelarch(input_ids, attention_mask)
#             loss = criterion(logits, labels.float())
#             total_loss += loss.item()
#             all_predictions.extend(predictions.tolist())
#             all_labels.extend(labels.squeeze(0).tolist())

#     # Compute accuracy and F1 score
#     accuracy = accuracy_score(all_labels, all_predictions)
#     f1 = f1_score(all_labels, all_predictions, average='macro')
#     val_loss = total_loss / len(validation_loader)
    
#     print(f"Accuracy: {accuracy}, F1 Score: {f1}, Loss : {val_loss}")
    


In [ ]:
model.eval()
all_predictions = []
all_labels = []
total_loss = 0
for input_ids, attention_mask, labels in test_loader:
        optimizer.zero_grad()
        logits = []
        for i,j in zip(input_ids, attention_mask):
            #print(i.squeeze(0).shape,j.squeeze(0).shape)
            logits.append(Modelarch(i.squeeze(1),j.squeeze(1)))
        logits = torch.stack(logits, dim=2).squeeze(1)
        labels = labels.float().squeeze(0)
        # print(logits)
        probabilities = F.softmax(logits, dim=1)
        # Get the predicted class index
        predictions = torch.argmax(probabilities, dim=1)
        predictions = F.one_hot(predictions, num_classes=4)
    
        # for i in range(len(labels)):
        #     if labels[i] == 1:
        #         label = i
        # for i in range(len(predictions)):
        #     if predictions[i] == 1:
        #         prediction = i
        # print(label, prediction)
        # print(input_ids[label])
        # print(type(label))
        # print(type(input_ids[label]))
    
        # print("correct: "+tokenizer.decode(input_ids[label].squeeze(), skip_special_tokens = True))
        # print("predicted: "+tokenizer.decode(input_ids[prediction].squeeze(), skip_special_tokens = True))
    
        # print(predictions)
        # print(labels)
        # logits = Modelarch(input_ids, attention_mask)
        loss = criterion(logits, labels.float())
        all_predictions.extend(predictions.tolist())
        all_labels.extend(labels.squeeze(0).tolist())
        total_loss += loss.item()
    
# Compute accuracy and F1 score
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


In [ ]:
torch.save(Modelarch, './Model2_6_45.pt')